In [18]:
import re
import numpy as np
import itertools
from collections import Counter
import linecache as lc

In [19]:
def read_file(filename):
    with open(filename, 'r') as file:
        content = file.read()  # Open the file in read mode

    data = {}  # Initialize data as an empty dictionary

    # Each file has number of inputs under which data is stored as a matrix
    numerical_data = re.findall(r'\[([\d\s.]+)\]', content)  # Extract values following each header as numerical values and remove '\t's and '\n's

    sections = ['Orders', 'Allocations', 'DistanceShelfShelf', 'DistancePackagingShelf', 'FullDistanceMatrixMetres']

    for i, section in enumerate(sections):
        data[section] = []
        lines = numerical_data[i].strip().split('\n')
        for line in lines:
            values = line.strip().split()  # Split using spaces
            data[section].append([int(val) for val in values])  # Assuming all values are integers

    return data

# Reading in data and running the entire function

In [20]:
filename = r"Xpress Data Files/Data_Xpress_FullDist_Metres.txt"
data = read_file(filename)

# Extracting data into arrays
Orders = data.get('Orders')
Allocations = np.asarray(data.get('Allocations')[0])
DistanceShelfShelf = np.asarray(data.get('DistanceShelfShelf'))
FullDistanceMatrix = np.asarray(data.get('FullDistanceMatrixMetres'))

NbShelves = 96
Shelves = range(1, NbShelves + 1)

FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 1)
FullDistanceMatrix = np.roll(FullDistanceMatrix, shift = 1, axis = 0)


# testing for allocation vector where every shelf is filled
allocations_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                    83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                    90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                    21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                    54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                    1, 33, 59, 42, 52,  45, 79, 39, 68, 73, 53]


# testing where only some of the extra shelves are filled
allocations_mid_full = [45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56, 
                        83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
                        90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
                        21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
                        54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
                        1, 33, 59, 42, 52,  45, 79,  0, 68,  0, 53]


In [21]:
Allocations

array([45, 79, 39, 68, 73, 53, 19, 44, 16, 71, 27, 41,  2, 46, 60, 67, 56,
       83, 80, 57, 69, 55, 75, 34, 89, 12, 81, 62, 23, 26, 24, 86,  3, 17,
       90, 58, 51, 25, 85, 65, 31, 11, 87, 10, 13, 70, 35, 32, 47,  6, 30,
       21, 43, 64, 66, 78, 76, 61,  8, 72, 22, 18, 82, 14, 28,  4,  5, 84,
       54, 48, 63, 29, 49, 74, 37, 36, 20, 38, 50,  7, 88,  9, 40, 77, 15,
        1, 33, 59, 42, 52,  0,  0,  0,  0,  0,  0])

In [22]:
def find_closest_product(current_shelf, order, distances):
    closest_shelf = None
    min_distance = float(1e6)
    for shelf in order:
        if distances[current_shelf][shelf] <= min_distance:
            closest_shelf = shelf
            min_distance = distances[current_shelf][shelf]
    return closest_shelf

def generate_order_lists(order):
    ''' 
    Takes a list with tuple elements and returns a list of lists 
    with all possible combinations of individual elements individual elements.
    '''
    order_lists = []
    tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
    for combination in itertools.product(*[order[i] for i in tuple_indices]):
        new_order = order.copy()
        for i, index in enumerate(tuple_indices):
            new_order[index] = combination[i]
        order_lists.append(new_order)
    return order_lists

def greedy_order_route(order, distances):
    ''' 
    Uses a greedy method of calculating the minimum distance. 

    Function has been split into two if statements to consider
    cases of orders where products are contained on more than one shelf.

    If products are contained on more than one shelf, the function constructs 
    a route with all possible shelf combinations and chooses the one with 
    the shortest distance.

    Returns a list containing the route and the total distance for the order.
    '''

    # If all products in the order are contained on one shelf only
    if not any(isinstance(product, tuple) for product in order):
        visited = [0]
        current_position = 0  
        for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
        visited.append(0)
        OrderDistance = 0
        for i in range(len(visited) - 1):
            OrderDistance += distances[visited[i]][visited[i+1]]
        order_distance_final = OrderDistance
        visited_final = visited

    # If one or more products in the order are contained on more than one shelf
    elif any(isinstance(product, tuple) for product in order):
        order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
        order_routes = []   # initialise a list of routes for all combinations                                 
        order_distances = [] # initialise a list of distances for all combinations
        
        # loop over all combinations
        for order in order_combinations:
            visited = [0]
            current_position = 0  
            for k in range(len(order)):
                closest_product = find_closest_product(current_position, order, distances)
                visited.append(closest_product)
                order.remove(closest_product)
                current_position = closest_product
            visited.append(0)
            order_routes.append(visited) # add the route for the combination to the list of routes
            OrderDistance = 0
            for i in range(len(visited) - 1):
                OrderDistance += distances[visited[i]][visited[i+1]]
            order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
        
        # select the order with the shortest distance among the combinations
        min_idx = order_distances.index(min(order_distances))
        visited_final = order_routes[min_idx] 
        order_distance_final = order_distances[min_idx]
            
    return visited_final, order_distance_final # return order route and distance

def convert_orders_to_shelf_indices(allocations):
    ''' 
    This function takes the allocation vector and returns an 
    order matrix with shelf indices instead of product indices.
    '''
    product_to_shelf = {}
    for shelf_index, product in enumerate(allocations):
        if product != 0:  # Check if the element is not zero
            if product not in product_to_shelf:
                product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
            else:
                # If the product already exists in the dictionary, append the new shelf index to the list
                product_to_shelf[product].append(shelf_index + 1)

    # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
    product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

    OrdersByShelf = []
    for order in Orders:
        order_shelf_indices = []
        for product_index in order:
            if product_index in product_to_shelf_tuples:
                shelf_indices = product_to_shelf_tuples[product_index]
                order_shelf_indices.append(shelf_indices)
            else:
                order_shelf_indices.append(0)  # Product not found in allocation matrix
        OrdersByShelf.append(order_shelf_indices)

    return OrdersByShelf

In [23]:
def q1_function(allocation_vector, distance_matrix):   

    OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector)
    TotalDistance = 0           # initialise counter for total distance
    DistancesPerOrder = []      # initalise list to contain the distances for each order 
    Routes = []                 # intialise list to contain the routes for each order
    for order in OrdersByShelf:
        visited_order_route, visited_order_dist = greedy_order_route(order, distance_matrix)
        Routes.append(visited_order_route)
        DistancesPerOrder.append(visited_order_dist)
        TotalDistance += visited_order_dist

    # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
    SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

    # The indices corresponding to the longest orders (descending order) 
    idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse = True)
    
    # List of routes ordered by distance (descending)
    OrderedRoutes = [Routes[i] for i in idx_longest_orders]

    return TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, OrderedRoutes

In [24]:
### TEST CASE ON CURRENT ALLOCATION ###
TotalDistance, DistancesPerOrder, SortedDistancesPerOrder, idx_longest_orders, OrderedRoutes = q1_function(Allocations)

#### Obtaining total distances for different allocations

In [25]:
print(f"Distance for the current allocation: {q1_function(Allocations)[0]} metres.")
print(f"Distance for all shelves filled: {q1_function(allocations_full)[0]} metres.")
print(f"Distance for some repeated products and some empty shelves: {q1_function(allocations_mid_full)[0]} metres.")

Distance for the current allocation: 290706 metres.
Distance for all shelves filled: 289200 metres.
Distance for some repeated products and some empty shelves: 289728 metres.


## The functions contained within the distance function (for testing and sanity checks)

In [26]:
# def find_closest_product(current_position, order, distances):
#     closest_shelf = None
#     min_distance = float(1e6)
#     for shelf in order:
#         if distances[current_position][shelf] <= min_distance:
#             closest_shelf = shelf
#             min_distance = distances[current_position][shelf]
#     return closest_shelf

# def generate_order_lists(order):
#     ''' 
#     Takes a list with tuple elements and returns a list of lists 
#     with all possible combinations of individual elements individual elements.
#     '''
#     order_lists = []
#     tuple_indices = [i for i, item in enumerate(order) if isinstance(item, tuple)]
#     for combination in itertools.product(*[order[i] for i in tuple_indices]):
#         new_order = order.copy()
#         for i, index in enumerate(tuple_indices):
#             new_order[index] = combination[i]
#         order_lists.append(new_order)
#     return order_lists

# def greedy_order_route(order, distances):
#     ''' 
#     Uses a greedy method of calculating the minimum distance. 

#     Function has been split into two if statements to consider
#     cases of orders where products are contained on more than one shelf.

#     If products are contained on more than one shelf, the function constructs 
#     a route with all possible shelf combinations and chooses the one with 
#     the shortest distance.

#     Returns a list containing the route and the total distance for the order.
#     '''

#     # If all products in the order are contained on one shelf only
#     if not any(isinstance(product, tuple) for product in order):
#         visited = [0]
#         current_position = 0  
#         for k in range(len(order)):
#                 closest_product = find_closest_product(current_position, order, distances)
#                 visited.append(closest_product)
#                 order.remove(closest_product)
#                 current_position = closest_product
#         visited.append(0)
#         OrderDistance = 0
#         for i in range(len(visited) - 1):
#             OrderDistance += distances[visited[i]][visited[i+1]]
#         order_distance_final = OrderDistance
#         visited_final = visited

#     # If one or more products in the order are contained on more than one shelf
#     elif any(isinstance(product, tuple) for product in order):
#         order_combinations = generate_order_lists(order) # create new orders with all possible combinations from tuples
#         order_routes = []   # initialise a list of routes for all combinations                                 
#         order_distances = [] # initialise a list of distances for all combinations
        
#         # loop over all combinations
#         for order in order_combinations:
#             visited = [0]
#             current_position = 0  
#             for k in range(len(order)):
#                 closest_product = find_closest_product(current_position, order, distances)
#                 visited.append(closest_product)
#                 order.remove(closest_product)
#                 current_position = closest_product
#             visited.append(0)
#             order_routes.append(visited) # add the route for the combination to the list of routes
#             OrderDistance = 0
#             for i in range(len(visited) - 1):
#                 OrderDistance += distances[visited[i]][visited[i+1]]
#             order_distances.append(OrderDistance) # add the distance for the combination to the list of distances
        
#         # select the order with the shortest distance among the combinations
#         min_idx = order_distances.index(min(order_distances))
#         visited_final = order_routes[min_idx] 
#         order_distance_final = order_distances[min_idx]
            
#     return visited_final, order_distance_final # return order route and distance

# def convert_orders_to_shelf_indices(allocations):
#     ''' 
#     This function takes the allocation vector and returns an 
#     order matrix with shelf indices instead of product indices.
#     '''
#     product_to_shelf = {}
#     for shelf_index, product in enumerate(allocations):
#         if product != 0:  # Check if the element is not zero
#             if product not in product_to_shelf:
#                 product_to_shelf[product] = [shelf_index + 1]  # Initialize with a list containing the current shelf index
#             else:
#                 # If the product already exists in the dictionary, append the new shelf index to the list
#                 product_to_shelf[product].append(shelf_index + 1)

#     # Convert product_to_shelf dictionary to a list of tuples if the product is assigned to multiple shelves
#     product_to_shelf_tuples = {k: tuple(v) if len(v) > 1 else v[0] for k, v in product_to_shelf.items()}

#     OrdersByShelf = []
#     for order in Orders:
#         order_shelf_indices = []
#         for product_index in order:
#             if product_index in product_to_shelf_tuples:
#                 shelf_indices = product_to_shelf_tuples[product_index]
#                 order_shelf_indices.append(shelf_indices)
#             else:
#                 order_shelf_indices.append(0)  # Product not found in allocation matrix
#         OrdersByShelf.append(order_shelf_indices)

#     return OrdersByShelf



# allocation_vector = allocations_full # change this to test different allocations

# OrdersByShelf = convert_orders_to_shelf_indices(allocation_vector) 
# TotalDistance = 0           # initialise counter for total distance
# DistancesPerOrder = []      # initalise list to contain the distances for each order 
# routes = []                 # intialise list to contain the routes for each order
# for order in OrdersByShelf:
#     visited_order_route, visited_order_dist = greedy_order_route(order, FullDistanceMatrix)
#     routes.append(visited_order_route)
#     DistancesPerOrder.append(3*visited_order_dist)
#     TotalDistance += 3*visited_order_dist

# # Replace DistancesPerOrder in the return statement with this if you want sorted distances to be returned
# SortedDistancesPerOrder = sorted(DistancesPerOrder, reverse=True)

# # The indices corresponding to the longest orders (descending order) 
# idx_longest_orders = sorted(range(len(DistancesPerOrder)), key=lambda i: DistancesPerOrder[i], reverse=True)

# Checking Construction Heuristics

In [27]:
def getMostCommonProducts(Orders):
    flattenedList = [x for x in [item for sublist in Orders for item in sublist] if x != 0]

    itemCounts = Counter(flattenedList)

    sorted_items = sorted(itemCounts.items(), key=lambda x: x[1], reverse=True)
    original_numbers = [item for item, _ in sorted_items]

    return itemCounts, original_numbers


def getClosestShelves(Distances):
    distanceToDoor = Distances[0] # consider the distance vector for the door
    indexed_numbers = list(enumerate(distanceToDoor)) # indices and distances from door

    sorted_indexed_numbers = sorted(indexed_numbers, key=lambda x: x[1]) # sort indices by distance from door (ascending)
    sorted_indexes = [tuple([index,dist])  for index, dist in sorted_indexed_numbers if index > 0]

    return sorted_indexes # print index of shelf and its corresponding distance


def ConstructionHeuristic_mostCommonOrder(Orders, Distances, fill_extra_shelves = True, nbAdditionalShelves=6):
    mostCommonProducts = getMostCommonProducts(Orders)[1] # get the most common products

    # if statement to decide whether you want to fill extra shelves or not
    # nbAdditionalShelves determines how many extra shelves you fill
    if fill_extra_shelves == True:
        mostCommonProducts = mostCommonProducts + mostCommonProducts[:nbAdditionalShelves] 
    elif fill_extra_shelves == False:
        mostCommonProducts = mostCommonProducts

    ClosestShelves, ClosestDistances = map(list, zip(*getClosestShelves(Distances)))

    newShelves = [0]*96  # initialise empty shelf allocation vector

    for i, prod in zip(ClosestShelves, mostCommonProducts):
        newShelves[i-1] = prod  # assign products based on the closest shelves

    return np.array(newShelves)

In [28]:
shelves, distances = map(list, zip(*getClosestShelves(FullDistanceMatrix)))

In [29]:
mostCommonOrderAllocation = ConstructionHeuristic_mostCommonOrder(Orders, 
                                                                  FullDistanceMatrix, 
                                                                  fill_extra_shelves=False, 
                                                                  nbAdditionalShelves=6)

In [30]:
q1_function(mostCommonOrderAllocation)[0]

205038

In [31]:
mostCommonOrderAllocation

array([34, 35, 18,  2, 50, 11, 48, 60, 19, 44, 47, 64, 74, 68, 69, 21, 88,
       14, 36, 53, 26,  3,  7,  0, 31, 71, 30, 58, 49, 28, 33, 43,  1, 59,
       66, 46, 65, 90, 16, 45, 27, 80, 75, 61, 40, 86, 42,  0, 70, 51, 67,
        4, 10, 52,  8, 63, 13, 72, 20, 32, 89, 82, 87, 24, 54,  9, 25, 62,
       73, 17,  0,  0, 15, 76,  6, 83, 38, 78, 55, 39, 57, 37, 84, 77, 79,
       29, 22,  5, 23, 56, 12, 41, 85, 81,  0,  0])

In [32]:
def allocation_to_txt(allocation_vector, file_path):
    np.savetxt(file_path, allocation_vector, fmt = "%d", delimiter = ",", newline=" ")

def allocation_from_txt(file):
    allocation_vector = lc.getline(file, 1).split(" ")[:-1]
    allocation_vector = [int(prod) for prod in allocation_vector]
    return np.array(allocation_vector)

In [33]:
allocation_to_txt(mostCommonOrderAllocation, "Allocation Vectors/mostCommonOrderAllocation.txt")

In [34]:
allocation_from_txt("Allocation Vectors/mostCommonOrderAllocation.txt")

array([34, 35, 18,  2, 50, 11, 48, 60, 19, 44, 47, 64, 74, 68, 69, 21, 88,
       14, 36, 53, 26,  3,  7,  0, 31, 71, 30, 58, 49, 28, 33, 43,  1, 59,
       66, 46, 65, 90, 16, 45, 27, 80, 75, 61, 40, 86, 42,  0, 70, 51, 67,
        4, 10, 52,  8, 63, 13, 72, 20, 32, 89, 82, 87, 24, 54,  9, 25, 62,
       73, 17,  0,  0, 15, 76,  6, 83, 38, 78, 55, 39, 57, 37, 84, 77, 79,
       29, 22,  5, 23, 56, 12, 41, 85, 81,  0,  0])

In [35]:
for i in range(6):
    allocation_extra_shelves = ConstructionHeuristic_mostCommonOrder(Orders,
                                                                     FullDistanceMatrix,
                                                                     fill_extra_shelves=True,
                                                                     nbAdditionalShelves=i+1)
    allocation_to_txt(allocation_extra_shelves, f"Allocation Vectors/CommonOrder_{i+1}_Extra")

In [45]:
results = q1_function(mostCommonOrderAllocation)

for i, order in enumerate(Orders):
    print(f"order {order} picked in dist {results[1][i]} taking route {results[4][i]}")

results[0]



order [50, 30, 0, 0, 0] picked in dist 36 taking route [0, 76, 77, 29, 10, 69, 0]
order [49, 18, 76, 0, 0] picked in dist 60 taking route [0, 3, 6, 74, 79, 19, 0]
order [72, 52, 51, 41, 35] picked in dist 150 taking route [0, 51, 52, 60, 30, 20, 0]
order [50, 4, 0, 0, 0] picked in dist 72 taking route [0, 27, 4, 30, 54, 46, 0]
order [76, 19, 26, 80, 6] picked in dist 162 taking route [0, 0, 4, 14, 76, 65, 0]
order [66, 67, 51, 84, 71] picked in dist 168 taking route [0, 27, 29, 78, 84, 46, 0]
order [39, 18, 29, 83, 49] picked in dist 132 taking route [0, 1, 5, 50, 82, 41, 0]
order [88, 2, 0, 0, 0] picked in dist 108 taking route [0, 0, 73, 32, 86, 68, 0]
order [63, 19, 0, 0, 0] picked in dist 120 taking route [0, 2, 30, 49, 80, 40, 0]
order [28, 11, 77, 8, 31] picked in dist 126 taking route [0, 25, 49, 10, 35, 87, 0]
order [32, 78, 38, 31, 44] picked in dist 150 taking route [0, 75, 61, 30, 8, 17, 0]
order [63, 31, 10, 68, 30] picked in dist 150 taking route [0, 73, 76, 61, 27, 41, 0]

205038